In [1]:
import tensorflow as tf

import pathlib
from typing import Any,Union

import numpy as np
import matplotlib.pyplot as plt

2022-09-28 14:10:08.468954: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-28 14:10:08.575245: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-28 14:10:09.076074: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-28 14:10:09.076116: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
def get_dataset_dir(path) -> str:
    return pathlib.Path(path)

In [3]:
BATCH_SIZE:int  = 32
IMG_HEIGHT:int  = 227
IMG_WIDTH:int   = 227

In [4]:
#import csv

MODEL_NAME = 'models/model_09.28.2022_12:33:15'
model:tf.keras.Model = tf.keras.models.load_model(MODEL_NAME)

2022-09-28 14:10:10.001087: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-28 14:10:10.025902: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-28 14:10:10.026141: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-28 14:10:10.026661: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
test_images = tf.keras.utils.image_dataset_from_directory(
    'data/giga_test',
    # validation_split=VAL_SPLIT,
    # subset="training",
    # seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    #batch_size=BATCH_SIZE
)

class_names = test_images.class_names
print(class_names)

Found 5000 files belonging to 2 classes.
['no-photo', 'photo']


In [6]:
i = 0
error = 0

for b_images, b_labels in test_images.take(-1):

    predictions = model.predict(b_images, verbose=0)
    
    for image, label, prediction in zip(b_images, b_labels, predictions) :
        
        if label.numpy() != int(np.round(prediction)):
            error += 1
            
            # plt.figure(figsize=(6,3))
            # plt.imshow(image.numpy().astype("uint8"))
            # plt.xlabel("Pred:{} {:2.0f}% (True:{})".format(class_names[int(np.round(prediction))], 100*np.max(predictions), class_names[label.numpy()]),color='red')

        i += 1
        

print(f"Number error: {error} on {i} images")
print(f"{error/i*100}% error")

2022-09-28 14:10:12.191278: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500
2022-09-28 14:10:12.580714: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Number error: 342 on 5000 images
6.84% error


In [7]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)